In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import scipy as sp
from bayes_opt import BayesianOptimization

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import GradientBoostingRegressor

# PolynomialFeatures

In [3]:
train_data=pd.read_csv('train_data_li.csv')
test_data=pd.read_csv('test_data_li.csv')

In [20]:
def lgb_eval(num_leaves,
             min_data,
             sub_feature,
             eta,
             learning_rate
            ):

    params['num_leaves'] = int(round(num_leaves))
    params['min_data'] = int(min_data)
    params['sub_feature'] = max(min(sub_feature, 1), 0)
    params['eta'] = float(eta)
    params['learning_rate'] = float(learning_rate)
    
    gbm = lgb.train(params, lgb_train, 4000)
    
    pred = gbm.predict(X_test)
    rmsetmp = sp.sqrt(sp.mean((y_test - pred) ** 2))
    score = 1 / (1 + rmsetmp)
    return score

In [22]:
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (20, 200),
                                        'min_data': (10, 80),
                                        'sub_feature': (0.3, 1),
                                        'eta':(0.1,0.9),
                                        'learning_rate':(0.001,0.1)
                                       })

In [23]:
if __name__ == '__main__':
    num_rounds = 4000
    random_state = 42
    num_iter = 25
    init_points = 5
    params = {
        'objective': 'regression',
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'min_hessian': 1,
        'verbose': -1
    }

In [15]:
best_submit=pd.read_csv('submit/result_20180810c_ensemble.csv',header=None)
best_submit.columns=['id','power_generation']

In [17]:
test_data['power_generation']=best_submit['power_generation']

In [24]:

train_test=pd.concat([train_data,test_data],axis=0)

In [25]:
train_test.reset_index(inplace=True)
del train_test['index']

In [26]:
X_train, y_train=train_test[500:].drop(['power_generation','ID'], axis=1),train_test[500:]['power_generation']
X_test, y_test=train_test[:500].drop(['power_generation','ID'], axis=1),train_test[:500]['power_generation']                                                                       

In [27]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_test = lgb.Dataset(X_test, y_test)

In [ ]:
#再次贝叶斯优化
lgbBO.maximize(init_points=init_points, n_iter=num_iter)

Initialization
-------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |       eta |   learning_rate |   min_data |   num_leaves |   sub_feature | 
    1 | 02m38s |    0.92497 |    0.3806 |          0.0614 |    24.6263 |     180.4317 |        0.9474 | 
    2 | 00m41s |    0.92822 |    0.3740 |          0.0988 |    70.9536 |      45.6755 |        0.3690 | 
    3 | 02m18s |    0.92399 |    0.5555 |          0.0039 |    56.7573 |     165.2152 |        0.9636 | 
    4 | 02m33s |    0.92064 |    0.6912 |          0.0434 |    48.5773 |     184.7577 |        0.9840 | 


In [ ]:
#加上伪标签找到最优参数
 2 | 00m41s |    0.92822 |    0.3740 |          0.0988 |    70.9536 |      45.6755 |        0.3690 | 

In [59]:
X_train, y_train=train_test.drop(['power_generation','ID'],axis=1),train_test['power_generation']
subdata=train_test[9000:].drop(['power_generation','ID'],axis=1)#测试数据

In [63]:
sub_data=train_test[9000:].drop(['power_generation','ID'],axis=1)

In [60]:
lgb_train = lgb.Dataset(X_train, y_train)
#subdata = lgb.Dataset(subdata)

In [61]:
params = {
        'eta':  0.3740,
        'learning_rate': 0.0988,
        'objective': 'regression',
        'boosting_type': 'gbdt',
        'metric': 'rmse',
        'min_hessian': 1,
        'verbose': -1,
        'num_leaves':70,
        'min_data':45,
        'sub_feature': 0.3690 
    
    }
gbm = lgb.train(params, lgb_train, 4000)

In [64]:
pred3 = gbm.predict(sub_data)

In [79]:

ensem=pd.read_csv('submit/result_20180810c_ensemble.csv',header=None)

In [81]:
ensem.columns=['id','power']

In [84]:
#对一下和线上最好成绩的差距
rmsetmp1 = sp.sqrt(sp.mean((ensem['power'].values - pred3) ** 2))
score1 = 1 / (1 + rmsetmp1)

In [86]:
rmsetmp1

0.037244933763835655

In [87]:
submit_data=pd.read_csv('submit_example.csv',header=None)
submit_data.columns=['ID','power_generation']

In [88]:
submit_data['power_generation']=pred3
submit_data.to_csv('submit/result_0817_lgbm_pesudo_1.csv',index=False, header=None) 